# Speculative Decoding

SGLang now provides an EAGLE-based speculative decoding option. The implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.

**Note:** Currently, Speculative Decoding in SGLang does not support radix cache.

### Performance Highlights

- Official EAGLE code ([SafeAILab/EAGLE](https://github.com/SafeAILab/EAGLE)): ~200 tokens/s
- Standard SGLang Decoding: ~156 tokens/s
- EAGLE Decoding in SGLang: ~297 tokens/s
- EAGLE Decoding in SGLang (w/ `torch.compile`): ~316 tokens/s

All benchmarks below were run on a single H100.

## EAGLE Decoding

To enable EAGLE-based speculative decoding, specify the draft model (`--speculative-draft`) and the relevant EAGLE parameters:

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algo EAGLE \
    --speculative-draft lmzheng/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-02-27 17:59:09] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, is_embedding=False, revision=None, skip_tokenizer_init=False, host='127.0.0.1', port=36968, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=True, tp_size=1, stream_interval=1, stream_output=False, random_seed=1048677179, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, enable_metr

[2025-02-27 17:59:29 TP0] Init torch distributed begin.
[2025-02-27 17:59:29 TP0] Load weight begin. avail mem=78.81 GB


[2025-02-27 17:59:29 TP0] The following error message 'operation scheduled before its operands' can be ignored.


[2025-02-27 17:59:30 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:00<00:00,  1.52it/s]


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.28s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.28s/it]

[2025-02-27 17:59:35 TP0] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=35.55 GB
[2025-02-27 17:59:35 TP0] KV Cache is allocated. K size: 0.16 GB, V size: 0.16 GB.
[2025-02-27 17:59:35 TP0] Memory pool end. avail mem=35.23 GB


[2025-02-27 17:59:36 TP0] max_total_num_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, max_running_requests=200, context_len=4096
[2025-02-27 17:59:36] INFO:     Started server process [3330328]
[2025-02-27 17:59:36] INFO:     Waiting for application startup.
[2025-02-27 17:59:36] INFO:     Application startup complete.
[2025-02-27 17:59:36] INFO:     Uvicorn running on http://127.0.0.1:36968 (Press CTRL+C to quit)


[2025-02-27 17:59:36] INFO:     127.0.0.1:59266 - "GET /v1/models HTTP/1.1" 200 OK


[2025-02-27 17:59:37] INFO:     127.0.0.1:59276 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-02-27 17:59:37 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-27 17:59:40] INFO:     127.0.0.1:59284 - "POST /generate HTTP/1.1" 200 OK
[2025-02-27 17:59:40] The server is fired up and ready to roll!


In [2]:
import openai

client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-02-27 17:59:41 TP0] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-27 17:59:41] INFO:     127.0.0.1:35560 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [3]:
terminate_process(server_process)

### EAGLE Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--cuda-graph-max-bs`:


In [4]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algo EAGLE \
    --speculative-draft lmzheng/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --cuda-graph-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-02-27 17:59:54] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-2-7b-chat-hf', chat_template=None, is_embedding=False, revision=None, skip_tokenizer_init=False, host='127.0.0.1', port=30167, mem_fraction_static=0.6, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=True, tp_size=1, stream_interval=1, stream_output=False, random_seed=1042655356, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, log_level='info', log_level_http=None, log_requests=False, show_time_cost=False, enable_metri

[2025-02-27 18:00:13 TP0] Init torch distributed begin.
[2025-02-27 18:00:13 TP0] Load weight begin. avail mem=63.19 GB


[2025-02-27 18:00:13 TP0] The following error message 'operation scheduled before its operands' can be ignored.


[2025-02-27 18:00:14 TP0] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.34s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.34s/it]

[2025-02-27 18:00:19 TP0] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=35.55 GB
[2025-02-27 18:00:19 TP0] KV Cache is allocated. K size: 0.16 GB, V size: 0.16 GB.
[2025-02-27 18:00:19 TP0] Memory pool end. avail mem=35.23 GB


[2025-02-27 18:00:19 TP0] max_total_num_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, max_running_requests=200, context_len=4096
[2025-02-27 18:00:19] INFO:     Started server process [3332868]
[2025-02-27 18:00:19] INFO:     Waiting for application startup.
[2025-02-27 18:00:19] INFO:     Application startup complete.
[2025-02-27 18:00:19] INFO:     Uvicorn running on http://127.0.0.1:30167 (Press CTRL+C to quit)


[2025-02-27 18:00:20] INFO:     127.0.0.1:56528 - "GET /v1/models HTTP/1.1" 200 OK


[2025-02-27 18:00:20] INFO:     127.0.0.1:56532 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-02-27 18:00:20 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-27 18:00:23] INFO:     127.0.0.1:56540 - "POST /generate HTTP/1.1" 200 OK
[2025-02-27 18:00:23] The server is fired up and ready to roll!


In [5]:
import openai

client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-02-27 18:00:25 TP0] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-02-27 18:00:25] INFO:     127.0.0.1:60028 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [6]:
terminate_process(server_process)